MODEL 

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [6]:
# Baca dataset dari file CSV
data = pd.read_excel('food_db.xlsx')

In [7]:
import json

def extract_ingredients(row):
    try:
        bahan = json.loads(row)
        return [item['bahan'] for item in bahan]
    except Exception as e:
        return []

data['Extracted Ingredients'] = data['Bahan'].apply(extract_ingredients)

print(data['Extracted Ingredients'])

0      [- 2 bh wortel, kupas, parut, - 2 sdm kismis, ...
1      [- 200 gr (2 ikat) baby kailan, potong-potong,...
2      [150 gr (3 ikat kecil) kangkung, bersihkan dan...
3      [- 200 gr fillet ayam, iris tipis memanjang, -...
4      [- 150 gr daging ayam tanpa tulang, potong-pot...
                             ...                        
983    [Cabe rawit : 2, Cabe keriting: 3, Jeruk nipis...
984    [Gyoza : 3 pcs, Nasi putih : 300 g, Cabai rawi...
985    [Gyoza : 5 pcs, Jamur Shimeji : 50 g, Bawang b...
986    [Gyoza : 3 pcs, Mie rebus instan : 1 bungkus, ...
987    [Gyoza : 3 pcs, Bakso Sapi : 8 buah, Tahu Baks...
Name: Extracted Ingredients, Length: 988, dtype: object


In [8]:
# Mengisi nilai yang hilang dengan 0
data.fillna(0, inplace=True)

# Mengubah daftar bahan menjadi string
data['Extracted Ingredients'] = data['Extracted Ingredients'].apply(lambda x: [ingredient.replace('-', '') for ingredient in x])
data['Ingredients String'] = data['Extracted Ingredients'].apply(lambda x: ', '.join(x))

# data['Ingredients String']

In [9]:
from sklearn.preprocessing import MultiLabelBinarizer
# Menggunakan MultiLabelBinarizer untuk encoding variabel "Ingredients String"
mlb = MultiLabelBinarizer()
bahan_encoded = mlb.fit_transform(data['Ingredients String'])

In [10]:
# Menggunakan OneHotEncoder untuk encoding variabel "Bahan"
bahan = data['Bahan'].values.reshape(-1, 1)  # Reshape to (n_samples, 1)
encoder = OneHotEncoder(sparse=False)
bahan_encoded = encoder.fit_transform(bahan)

# bahan_encoded

e:\ProgramFiles\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [11]:
# Membaca atribut gizi sebagai fitur
features = data[['Kalori', 'Protein', 'Karbo', 'Lemak', 'Serat']].values

# Normalisasi fitur
features_norm = (features - features.mean(axis=0)) / features.std(axis=0)

# Membaca atribut rekomendasi sebagai target
target = data['Makanan'].astype('category').cat.codes

# Menggabungkan -0--- dan variabel "Bahan" yang telah di-encode
features_encoded = np.concatenate((features_norm, bahan_encoded), axis=1)

# Membagi data menjadi data pelatihan dan data pengujian
X_train, X_test, y_train, y_test = train_test_split(features_norm, target, test_size=0.2, random_state=42)

# Membangun model TensorFlow
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(5,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(data['Makanan'].unique()), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

# Melatih model
model.fit(features_norm, target, epochs=50, batch_size=1)



Epoch 1/50
988/988 [==============================] - 6s 3ms/step - loss: 7.0772
Epoch 2/50
988/988 [==============================] - 2s 2ms/step - loss: 6.8363
Epoch 3/50
988/988 [==============================] - 2s 2ms/step - loss: 6.1759
Epoch 4/50
988/988 [==============================] - 2s 2ms/step - loss: 5.4375
Epoch 5/50
988/988 [==============================] - 2s 2ms/step - loss: 4.7614
Epoch 6/50
988/988 [==============================] - 2s 2ms/step - loss: 4.2057
Epoch 7/50
988/988 [==============================] - 2s 2ms/step - loss: 3.7522
Epoch 8/50
988/988 [==============================] - 2s 2ms/step - loss: 3.3887
Epoch 9/50
988/988 [==============================] - 2s 2ms/step - loss: 3.0944
Epoch 10/50
988/988 [==============================] - 2s 2ms/step - loss: 2.8674
Epoch 11/50
988/988 [==============================] - 2s 2ms/step - loss: 2.6701
Epoch 12/50
988/988 [==============================] - 2s 2ms/step - loss: 2.5327
Epoch 13/50
988/988 [====

In [12]:
# Evaluasi model dengan data pengujian
loss = model.evaluate(X_test, y_test, verbose=0)

print("Loss: ", loss)


Loss:  0.8062357306480408


In [13]:
# menyimpan model
model.save('my_model.h5')

In [14]:
# fungsi tambahan
# function baru (di cek dulu alergi baru dimasukkan di list)
def get_top_5_recommendations(nutrition_data, allergies=[]):
    # Normalisasi data gizi pengguna
    nutrition_data_norm = (nutrition_data - features.mean(axis=0)) / features.std(axis=0)

    # Memprediksi probabilitas kelas makanan
    predictions = model.predict(np.array([nutrition_data_norm]))
    sorted_predictions = np.argsort(predictions[0])[::-1]
    # Mendapatkan indeks 5 kelas dengan probabilitas tertinggi
    # top_5_indices = np.argsort(predictions[0])[::-1][:5]
    recommended_foods = []

    for index in sorted_predictions:
        # Getting the food name
        food = data.loc[index, 'Makanan']

        # Checking if the food contains any allergies
        if any(allergy in food for allergy in allergies):
            continue  # Skip this food if it contains allergens

        # Adding the food to the recommended foods list
        recommended_foods.append(food)

        # Break the loop if we have 5 recommended foods
        if len(recommended_foods) == 5:
            break

    return recommended_foods

def calculate_bmr(weight, height, age, gender):
    if gender == 'male':
        bmr = 10 * weight + 6.25 * height - 5 * age + 5
    else:
        bmr = 10 * weight + 6.25 * height - 5 * age - 161
    return bmr

In [15]:
# Contoh penggunaan
weight = 80  # Ganti dengan berat badan pengguna dalam kilogram
height = 180  # Ganti dengan tinggi badan pengguna dalam sentimeter
age = 21  # Ganti dengan usia pengguna
gender = 'Male'  # Ganti dengan jenis kelamin pengguna ('Male' atau 'Female')
allergies = ['Ayam', 'Tauge']  # Ganti dengan daftar alergi pengguna, kosongkan jika tidak ada

bmr = calculate_bmr(weight, height, age, gender)
nutrition_data = [bmr]  # Gunakan BMR sebagai kalori makanan yang direkomendasikan

# Mendapatkan rekomendasi makanan
recommended_foods = get_top_5_recommendations(nutrition_data, allergies)
print("Rekomendasi makanan:")
for food in recommended_foods:
    print(food)

1/1 [==============================] - 1s 543ms/step
Rekomendasi makanan:
Fu Yung Hai Asam Manis
Sweet Sour Salad Gyoza
Sup Udang Jamur Merdeka
Mie Lidi Merdeka
Nasi Ulam Betawi
